In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import random
from datasets import load_dataset
from collections import defaultdict
import tqdm
from datetime import datetime, timedelta
import logging

logging.basicConfig(level=logging.INFO)

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)
ds = dataset["full"]

# Use a fixed cutoff date (e.g., January 1, 2024)
FIXED_CUTOFF_DATE = datetime(2024, 1, 1)
DATE_CUTOFF = FIXED_CUTOFF_DATE - timedelta(days=3*365) 

In [ ]:
def convert_timestamp(timestamp):
    return datetime.fromtimestamp(timestamp / 1000)

logging.info("Filtering dataset for time cutoff")
filtered_ds = [d for d in tqdm.tqdm(ds) if convert_timestamp(d['timestamp']) > DATE_CUTOFF]

# Organize data by user_id
logging.info("Organizing data by user_id")
d_dic = defaultdict(list)
for d in filtered_ds:
    d_dic[d["user_id"]].append(d)

# Print the number of user profiles in the filtered dataset
num_user_profiles = len(d_dic)
logging.info(f"Number of user profiles in the filtered dataset: {num_user_profiles}")
print(f"Number of user profiles in the filtered dataset: {num_user_profiles}")

# Calculate the distribution of user profile sizes in the main dataset
user_profile_sizes = {user_id: len(reviews) for user_id, reviews in d_dic.items()}
user_profile_df = pd.DataFrame(list(user_profile_sizes.items()), columns=['user_id', 'profile_size'])
filtered_profile_size_distribution = user_profile_df['profile_size'].value_counts(normalize=True).sort_index()

# Print the distribution of profile sizes in the filtered dataset
logging.info("Profile size distribution in the filtered dataset:")
print(filtered_profile_size_distribution)
print(len(filtered_profile_size_distribution))


In [ ]:
# Calculate average review length size
logging.info("Calculating average review length size")
total_review_length = 0
total_reviews = 0

for d in filtered_ds:
    review_length = len(d['text'])  # Length of each review text
    total_review_length += review_length
    total_reviews += 1

if total_reviews > 0:
    average_review_length = total_review_length / total_reviews
else:
    average_review_length = 0

logging.info(f"Total number of reviews: {total_reviews}")
logging.info(f"Average review length: {average_review_length}")
print(f"Total number of reviews: {total_reviews}")
print(f"Average review length: {average_review_length}")

In [ ]:
# Calculate the distribution of reviews per product (product-review distribution)
product_review_counts = defaultdict(int)
for review in filtered_ds:
    product_review_counts[review['asin']] += 1

def print_product_review_distribution_with_percentages(local_product_counts, split_name):
    # Calculate the distribution of review counts
    review_count_distribution = defaultdict(int)
    
    for asin, count in local_product_counts.items():
        review_count_distribution[count] += 1
    
    total_products = sum(review_count_distribution.values())
    print(f"Review Count Distribution for {split_name} split:")
    sorted_review_distribution = sorted(review_count_distribution.items(), key=lambda x: x[0])
    
    for review_count, product_count in sorted_review_distribution:
        percentage = (product_count / total_products) * 100
        print(f"Products with {review_count} reviews: {product_count} ({percentage:.2f}%)")
    print("\n")

print_product_review_distribution_with_percentages(product_review_counts, "Filtered Dataset")

In [ ]:
# Count global neighbors for the first review in each user's profile
global_neighbor_counts = defaultdict(int)

#Need to edit this to adjust with random review neighbor global count
# Go through the filtered dataset and focus on the first review for each user
for user_id, user_reviews in d_dic.items():
    first_review_asin = user_reviews[0]['asin']  # Focus on the first review
    global_neighbor_counts[first_review_asin] += 1  # Increment count for this product globally

def print_global_neighbor_distribution_with_percentages(global_neighbor_counts, split_name):
    # Calculate the distribution of neighbor counts
    neighbor_count_distribution = defaultdict(int)
    
    for asin, count in global_neighbor_counts.items():
        neighbor_count_distribution[count] += 1
    
    # Calculate total number of products
    total_products = sum(neighbor_count_distribution.values())
    
    # Print the distribution with percentages
    print(f"Global Neighbor Count Distribution for {split_name} split (based on first review):")
    sorted_neighbor_distribution = sorted(neighbor_count_distribution.items(), key=lambda x: x[0])
    
    for neighbor_count, product_count in sorted_neighbor_distribution:
        percentage = (product_count / total_products) * 100
        print(f"Products with {neighbor_count} reviews: {product_count} ({percentage:.2f}%)")
    print("\n")

print_global_neighbor_distribution_with_percentages(global_neighbor_counts, "Global Neighbors (First Review)")

In [ ]:
# Number of users per set
num_users_train = 20000  # Set the desired number of users in the training set
num_users_dev = 2500
num_users_test = 2500

# Convert dictionary to list of users
users_list = list(d_dic.items())
    
# Build global product counts across the entire dataset
def build_global_product_counts(users):
    product_count = defaultdict(list)  # Store list of users who reviewed each product
    for user_id, reviews in users:
        # Focus on the first review in each user's profile
        first_review = reviews[0]
        product_count[first_review['asin']].append(user_id)  # Track which users reviewed the first product
    return product_count

# Global product counts for the entire dataset
global_product_counts = build_global_product_counts(users_list)

# Function to pick a random review that has global neighbors, or exclude the user if none do
def set_random_first_review_with_global_neighbor(user_reviews, global_product_counts):
    # Filter reviews that have at least one global neighbor
    valid_reviews = [review for review in user_reviews if len(global_product_counts[review['asin']]) > 1]
    
    if valid_reviews:
        random_review = random.choice(valid_reviews)  # Pick a random review with global neighbors
        user_reviews.remove(random_review)
        user_reviews.insert(0, random_review)
    else:
        return None  # No reviews with global neighbors, exclude the user
    
    return user_reviews

# Helper function to add a user to a specified split
def add_user_to_split(user_id, user_reviews, split_users, local_product_counts, added_users):
    # Add the user to the split
    split_users.append((user_id, user_reviews))
    added_users.add(user_id)
    # Update local product counts for the split
    for review in user_reviews:
        local_product_counts[review['asin']] += 1

# Function to distribute global neighbor groups across train, dev, and test splits with local neighbor check
def distribute_global_neighbors(users, num_users_train, num_users_dev, num_users_test, global_product_counts):
    train_users = []
    dev_users = []
    test_users = []
    
    # Initialize local product counts for train, dev, and test splits
    local_product_counts_train = defaultdict(int)
    local_product_counts_dev = defaultdict(int)
    local_product_counts_test = defaultdict(int)
    
    # Track which users have already been added
    added_users = set()

    # Exclude products with only one review
    filtered_product_list = [asin for asin, user_ids in global_product_counts.items() if len(user_ids) > 1]

    # Set the seed before shuffling to ensure reproducibility
    random.seed(RANDOM_SEED)
    random.shuffle(filtered_product_list)

    # Go through each product's global neighbors in a random order
    for asin in filtered_product_list:
        user_ids = global_product_counts[asin]

        # Set the seed before shuffling users to ensure reproducibility
        random.seed(RANDOM_SEED)
        random.shuffle(user_ids)

        # Add the first two users to the training set
        if len(train_users) < num_users_train - 1:
            for user_id in user_ids[:2]:  # Add the first two users to the training set
                if user_id in added_users:
                    continue  # Skip users who have already been added

                # Get the user's reviews
                user_reviews = [user for user in users if user[0] == user_id][0][1]

                # Add the user to the training split
                add_user_to_split(user_id, user_reviews, train_users, local_product_counts_train, added_users)

        # Add the next two users to the dev set if training set is filled
        elif len(dev_users) < num_users_dev - 1:
            for user_id in user_ids[:2]:  # Add the next two users to the dev set
                if user_id in added_users:
                    continue  # Skip users who have already been added

                # Get the user's reviews
                user_reviews = [user for user in users if user[0] == user_id][0][1]

                # Add the user to the dev split
                add_user_to_split(user_id, user_reviews, dev_users, local_product_counts_dev, added_users)

        # Add the next two users to the test set if dev set is filled
        elif len(test_users) < num_users_test - 1:
            for user_id in user_ids[:2]:  # Add the next two users to the test set
                if user_id in added_users:
                    continue  # Skip users who have already been added

                # Get the user's reviews
                user_reviews = [user for user in users if user[0] == user_id][0][1]

                # Add the user to the test split
                add_user_to_split(user_id, user_reviews, test_users, local_product_counts_test, added_users)

        # Add subsequent users for this product, ensuring they have local neighbors
        for i, user_id in enumerate(user_ids[2:], start=2):
            if user_id in added_users:
                continue  # Skip users who have already been added

            # Get the user's reviews
            user_reviews = [user for user in users if user[0] == user_id][0][1]

            # Ensure subsequent users have local neighbors in the train split
            if len(train_users) < num_users_train:
                if local_product_counts_train[asin] > 0:  # Ensure local neighbor exists
                    add_user_to_split(user_id, user_reviews, train_users, local_product_counts_train, added_users)

            # Ensure subsequent users have local neighbors in the dev split
            elif len(dev_users) < num_users_dev:
                if local_product_counts_dev[asin] > 0:  # Ensure local neighbor exists
                    add_user_to_split(user_id, user_reviews, dev_users, local_product_counts_dev, added_users)

            # Ensure subsequent users have local neighbors in the test split
            elif len(test_users) < num_users_test:
                if local_product_counts_test[asin] > 0:  # Ensure local neighbor exists
                    add_user_to_split(user_id, user_reviews, test_users, local_product_counts_test, added_users)
            
            # Stop if all splits are full
            if len(train_users) >= num_users_train and len(dev_users) >= num_users_dev and len(test_users) >= num_users_test:
                break

    return train_users, dev_users, test_users, local_product_counts_train, local_product_counts_dev, local_product_counts_test


In [ ]:
train_users, dev_users, test_users, local_product_counts_train, local_product_counts_dev, local_product_counts_test = distribute_global_neighbors(
    users_list, num_users_train, num_users_dev, num_users_test, global_product_counts
)

# Check the number of users added to each set
logging.info(f"Number of users in the test set: {len(test_users)}")
logging.info(f"Number of users in the dev set: {len(dev_users)}")

In [ ]:
# Function to get the profile size distribution
def get_profile_size_distribution(users):
    profile_sizes = [len(reviews) for user_id, reviews in users]
    return pd.Series(profile_sizes).value_counts().sort_index()

# Function to calculate the local neighbor count for each user's first review in the split
def get_local_neighbor_count(users, local_product_counts):
    user_neighbor_counts = []
    
    for user_id, reviews in users:
        first_review = reviews[0]  # Focus on the first review in the profile
        asin = first_review['asin']
        local_neighbors = local_product_counts[asin] - 1  # Subtract 1 to exclude the current review
        user_neighbor_counts.append((user_id, local_neighbors))
    
    return pd.Series([count for _, count in user_neighbor_counts]).value_counts().sort_index()

# Function to print the local neighbor count distribution with percentages
def print_local_neighbor_distribution_with_percentages(local_neighbor_count, split_name):
    total_users = local_neighbor_count.sum()  # Total number of users

    # Print the distribution with percentages
    print(f"{split_name} Local Neighbor Count Distribution (for the first review):")
    for neighbor_count, count in local_neighbor_count.items():
        percentage = (count / total_users) * 100
        print(f"Users with {neighbor_count} neighbors: {count} ({percentage:.2f}%)")
    print("\n")

# Function to print the profile size distribution with percentages
def print_profile_size_distribution_with_percentages(profile_size_distribution, split_name):
    total_users = profile_size_distribution.sum()  # Total number of users

    # Print the distribution with percentages
    print(f"{split_name} Profile Size Distribution:")
    for profile_size, count in profile_size_distribution.items():
        percentage = (count / total_users) * 100
        print(f"Users with profile size {profile_size}: {count} ({percentage:.2f}%)")
    print("\n")

# Calculate the local neighbor count for each split (for the first review only)
train_local_neighbor_count = get_local_neighbor_count(train_users, local_product_counts_train)
dev_local_neighbor_count = get_local_neighbor_count(dev_users, local_product_counts_dev)
test_local_neighbor_count = get_local_neighbor_count(test_users, local_product_counts_test)

# Calculate the profile size distributions for the train, dev, and test sets
train_profile_size_distribution = get_profile_size_distribution(train_users)
dev_profile_size_distribution = get_profile_size_distribution(dev_users)
test_profile_size_distribution = get_profile_size_distribution(test_users)

# Print local neighbor count distributions with percentages for each split
print_local_neighbor_distribution_with_percentages(train_local_neighbor_count, "Train")
print_local_neighbor_distribution_with_percentages(dev_local_neighbor_count, "Dev")
print_local_neighbor_distribution_with_percentages(test_local_neighbor_count, "Test")

# Print profile size distributions with percentages for each split
print_profile_size_distribution_with_percentages(train_profile_size_distribution, "Train")
print_profile_size_distribution_with_percentages(dev_profile_size_distribution, "Dev")
print_profile_size_distribution_with_percentages(test_profile_size_distribution, "Test")


In [ ]:
def print_product_review_distribution_with_percentages(local_product_counts, split_name):
    # Calculate the distribution of review counts
    review_count_distribution = defaultdict(int)
    
    for asin, count in local_product_counts.items():
        review_count_distribution[count] += 1
    
    # Calculate total number of products
    total_products = sum(review_count_distribution.values())
    
    # Print the distribution with percentages
    print(f"Review Count Distribution for {split_name} split:")
    sorted_review_distribution = sorted(review_count_distribution.items(), key=lambda x: x[0])
    
    for review_count, product_count in sorted_review_distribution:
        percentage = (product_count / total_products) * 100
        print(f"Products with {review_count} reviews: {product_count} ({percentage:.2f}%)")
    print("\n")
    
print_product_review_distribution_with_percentages(local_product_counts_train, "Train")
print_product_review_distribution_with_percentages(local_product_counts_dev, "Dev")
print_product_review_distribution_with_percentages(local_product_counts_test, "Test")

In [ ]:
import json
import tqdm

# Function to process user data (unchanged, but you can modify if needed)
def process_user_data(users):
    out = []
    predictions = []
    for user_id, reviews in tqdm.tqdm(users):
        # Generate user input and output (as done before)
        user_entry = {"id": user_id}
        pred_entry = {"id": user_id, "output": reviews[0]['title']}

        # Add full user profile to user_entry
        user_entry["profile"] = [
            {
                'rating': d['rating'], 
                'title': d['title'],
                'text': d['text'],
                # Uncomment the lines below if you want to include these fields
                # 'timestamp': d['timestamp'],
                # 'helpful_vote': d['helpful_vote'],
                # 'verified_purchase': d['verified_purchase'],
                 "productAsin": d['asin']
            }
            for d in reviews
        ]

        # Append entries to output lists
        out.append(user_entry)
        predictions.append(pred_entry)
    return out, predictions

# Logging info
logging.info("Processing train, test, and dev data")

# Process the train, test, and dev sets
out_train, predictions_train = process_user_data(train_users)
out_test, predictions_test = process_user_data(test_users)
out_dev, predictions_dev = process_user_data(dev_users)

# File name base (you can modify this as needed)
file_name_base = "amazon_title_generation"

# Function to write data to files
def write_to_file(data, file_suffix):
    file_name = f"{file_name_base}_{file_suffix}.json"
    with open(file_name, "w") as file:
        json.dump(data, file, indent=4)

# Logging and writing the files
logging.info("Writing data to files")
write_to_file(out_train, "questions_train2")
write_to_file(predictions_train, "outputs_train2")
write_to_file(out_test, "questions_test2")
write_to_file(predictions_test, "outputs_test2")
write_to_file(out_dev, "questions_dev2")
write_to_file(predictions_dev, "outputs_dev2")

# Log completion
logging.info("Data successfully written to files.")
